In [29]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if "Aquifer_Petrignano" in filename:
            print(os.path.join(dirname, filename))

In [30]:
pip install fbprophet

# EDA

### Import Libraries

In [31]:
import pandas as pd
from pandas.plotting import autocorrelation_plot
import numpy as np
import matplotlib.pyplot as plt #visualization
import seaborn as sns  #visualization
%matplotlib inline


from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from fbprophet import Prophet

from datetime import datetime, date
import math
import warnings
warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize': (20,15)})

#### TimeSeries : Any dataset that is a function of time 

### Read File

In [32]:
df = pd.read_csv('/kaggle/input/acea-water-prediction/Aquifer_Petrignano.csv')
df.head()

In [33]:
df.describe().T

In [34]:
df.info()

### Cleaning

In [35]:
# Remove NA values
df = df[df['Rainfall_Bastia_Umbra'].notna()].reset_index(drop = True)
# remove columns that are not usefull
df = df.drop(['Depth_to_Groundwater_P24', 'Temperature_Petrignano'],axis = 1)


In [36]:
# change column names
df.columns = ['date', 'rainfall', 'depth_to_groundwater', 'temperature', 'drainage_volume', 'river_hydrometry']

In [37]:
# Convert date to datatime object
df['date']= pd.to_datetime(df['date'], format = '%d/%m/%Y')

In [38]:
# Assign the output and also the input features
target = df['depth_to_groundwater']
features = df.drop('depth_to_groundwater',axis = 1)

#### Visualization

In [39]:
plt.figure(figsize = (10,10))
sns.scatterplot(x = 'drainage_volume', y = 'depth_to_groundwater', data = df)

In [40]:
## See how the features (rainfall,temperature etc.) change with date
f,ax = plt.subplots(nrows = 5, ncols = 1, figsize = (15,25))

for i, column in enumerate(df.drop('date',axis = 1).columns):
    
    sns.lineplot(x = df['date'], y = df[column].fillna(method = 'ffill'), ax = ax[i])
    
    ax[i].set_title(f' Features: {column}')



In [41]:
df.isnull().sum()

In [42]:
sns.heatmap(df.isna(), cmap = 'Blues')

#### important step while dealing with timeseries is that, it has to be in chronological order and also equidistant whatever the interval chosen

In [43]:
# sort df by date and check if its equidistant
df = df.sort_values(by = 'date')

df['Delta'] = df['date'] - df['date'].shift(1)
df['Delta'].value_counts()

#### Ways to deal with null values in numeric features.
- Replace with Mean values
- ffill and bfill to fill values with the forward and last value
- linearly interpolated values by using .interpolate()
- Drop the rows ( only an option if the dataset is large and the # of null rows are small)

In [44]:
# To get rid of the null values, interpolate the values

df['drainage_volume'] = df['drainage_volume'].interpolate()
df['river_hydrometry'] = df['river_hydrometry'].interpolate()
df['depth_to_groundwater'] = df['depth_to_groundwater'].interpolate()

### Resampling 

In [45]:
df.head()

#### till now we have,
- sorted the values by date
- made the data eqidistant
- handled missing data by removing few rows and then interpolating rest of the nan values by comparing 
- resample the data to smooth the curve

#### Visialization of different resampled data

In [46]:
f, ax = plt.subplots(3,2, figsize = (15,10))

sns.lineplot(x = df['date'],y =df['temperature'], ax = ax[0,0], color = 'darkorange' )
ax[0,0].set_title("Original temperature", fontsize = 14)

resampled_temp = df[['date','temperature']].resample('7D', on = 'date').sum().reset_index(drop = False)

sns.lineplot(x = resampled_temp['date'],y =resampled_temp['temperature'], ax = ax[1,0], color = 'dodgerblue' )
ax[1,0].set_title("Weekly temperature", fontsize = 14)

resampled_tempM = df[['date','temperature']].resample('M', on = 'date').sum().reset_index(drop = False)

sns.lineplot(x = resampled_tempM['date'],y =resampled_tempM['temperature'], ax = ax[2,0], color = 'dodgerblue' )
ax[1,0].set_title("Monthly temperature", fontsize = 14)

sns.lineplot(df['date'], df['drainage_volume'], color='dodgerblue', ax=ax[0, 1])
ax[0, 1].set_title('Drainage Volume', fontsize=14)

resampled_df = df[['date','drainage_volume']].resample('7D', on='date').sum().reset_index(drop=False)
sns.lineplot(resampled_df['date'], resampled_df['drainage_volume'], color='dodgerblue', ax=ax[1, 1])
ax[1, 1].set_title('Weekly Drainage Volume', fontsize=14)

resampled_df = df[['date','drainage_volume']].resample('M', on='date').sum().reset_index(drop=False)
sns.lineplot(resampled_df['date'], resampled_df['drainage_volume'], color='dodgerblue', ax=ax[2, 1])
ax[2, 1].set_title('Monthly Drainage Volume', fontsize=14)
plt.tight_layout()


In [47]:
# Downsample
df_downsample = df[['date',
                 'depth_to_groundwater', 
                 'temperature',
                 'drainage_volume', 
                 'river_hydrometry',
                 'rainfall']].resample('7D', on ='date').mean().reset_index(drop = False)

In [48]:
df = df_downsample.copy()

#### Check Stationarity

#### To use timeseries models like ARIMA, we assume stationarity in data, which means the mean and variance is constant

#### The tests can be done with few methods:

- visually: check for trends and seasoanlity
- basic statistics: partition the data and check for mean and variance
- statistical test: Augmented Dickery Fuller test

In [49]:
## visually check to see if the mean and variance is constant (stationary)

rolling_window = 52 # there are 52 weeks in 1 year

f, ax = plt.subplots(2,1,figsize = (15,12))

sns.lineplot(x = df['date'], y = df['drainage_volume'], ax = ax[0], color = 'dodgerblue')
sns.lineplot(x = df['date'], y = df['drainage_volume'].rolling(rolling_window).mean(), ax = ax[0], color = 'darkorange', label = "rolling mean")
sns.lineplot(x = df['date'], y = df['drainage_volume'].rolling(rolling_window).std(), ax = ax[0], color = 'black', label = "rolling STD")
ax[0].set_title('Depth to Groundwater: Non-stationary \nnon-constant mean & non-constant variance',fontsize = 14)

sns.lineplot(x = df['date'], y = df['temperature'], ax = ax[1], color = 'dodgerblue')
sns.lineplot(x = df['date'], y = df['temperature'].rolling(rolling_window).mean(), ax = ax[1], color = 'darkorange',label = "rolling mean")
sns.lineplot(x = df['date'], y = df['temperature'].rolling(rolling_window).std(), ax = ax[1], color = 'black', label = "rolling STD")
ax[1].set_title('Temperature: Non-stationary \nvariance is time-dependent (seasonality)', fontsize=14)

plt.tight_layout()
plt.show()

#### visually, we can see that it is not stationary

#### basically if the p-value is less than 5%, then it is not statistically significant which means we have to agree null hypothesis, which means it is non-stationary

**p-value > significance level (default: 0.05)**: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
**p-value <= significance level (default: 0.05)**: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.
On the other hand, the null hypothesis can be rejects if the test statistic is less than the critical value.

**ADF statistic > critical value**: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
**ADF statistic < critical value**: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

#### Adfuller Test

In [50]:
result = adfuller(df['depth_to_groundwater'].values)
result

#### To use the ARIMA model, we need to transform the non- stationary model to stationary model,we can do that in two different ways:

- Transformation: log or square root to stabalize non-constant variance
- differencing: Subtracts the current value from the previous

#### So, since it is non-stationary suggested using the visual representation and adfuller method, we need to make it stationary.

#### Transformation

In [51]:
df['depth_to_groundwater_log'] = np.log(abs(df['depth_to_groundwater']))
f = plt.figure()
axes = f.add_axes([1,1,1,1])
axes.plot(df['depth_to_groundwater_log'].values)
#sns.lineplot(x = df['date'], y = df['depth_to_groundwater_log'], ax = ax[1], color = 'darkorange')

## Feature Engineering

In [52]:
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
df['day_of_year'] = pd.DatetimeIndex(df['date']).dayofyear
df['week_of_year'] = pd.DatetimeIndex(df['date']).weekofyear
df['quarter'] = pd.DatetimeIndex(df['date']).quarter
df['season'] = df['month'] % 12 // 3 + 1

df[['date', 'year', 'month', 'day', 'day_of_year', 'week_of_year', 'quarter', 'season']].head()

In [53]:
month_in_year = 12
df['month_sin'] = np.sin(2*np.pi*df['month']/month_in_year)
df['month_cos'] = np.cos(2*np.pi*df['month']/month_in_year)

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))

sns.scatterplot(x=df.month_sin, y=df.month_cos, color='dodgerblue')
plt.show()

## TimeSeries Decomposition

In [54]:

core_columns = ['depth_to_groundwater', 'temperature', 'drainage_volume',
       'river_hydrometry', 'rainfall']

for column in core_columns:
    decomp = seasonal_decompose(df[column],model = 'additive',period = 52, extrapolate_trend = 'freq')
    df[f"{column}_trend"] = decomp.trend
    df[f"{column}_seasonal"] = decomp.seasonal
    

In [55]:
f,ax = plt.subplots(nrows = 4, ncols = 2, figsize = (16,8))

for i, column in enumerate(['temperature','depth_to_groundwater']):
    
    res = seasonal_decompose(x = df[column], model = 'additive', period = 52, extrapolate_trend = 'freq')
    
    ax[0,i].set_title(f"Decomposition of observed {column}")
    res.observed.plot(ax = ax[0,i],color = 'dodgerblue',legend = False)
    ax[0,i].set_ylabel("Observed", fontsize = 14)
    
    ax[1,i].set_title(f"Decomposition of trend {column}")
    res.trend.plot(ax = ax[1,i],color = 'dodgerblue',legend = False)
    ax[1,i].set_ylabel("Trend", fontsize = 14)
    
    
    ax[2,i].set_title(f"Decomposition of seasonal {column}")
    res.seasonal.plot(ax = ax[2,i],color = 'dodgerblue',legend = False)
    ax[2,i].set_ylabel("Seasonal", fontsize = 14)
    
    ax[3,i].set_title(f"Decomposition of seasonal {column}")
    res.resid.plot(ax = ax[3,i],color = 'dodgerblue',legend = False)
    ax[3,i].set_ylabel("Residual", fontsize = 14)
    
plt.tight_layout()    

In [56]:
# some important functions used with pandas 
## dateframe.rolling().mean(), dataframe.resample().mean(), dataframe.shift()

In [57]:
weeks_in_month = 4 
df['temperature_seasonal_shift_b_2m'] = df['temperature_seasonal'].shift(-2 * weeks_in_month)

In [58]:
# lag calculation, to see the correlation with each other

weeks_in_month = 4

for column in core_columns:
    df[f'{column}_seasonal_shift_b_2m'] = df[f'{column}_seasonal'].shift(-2 * weeks_in_month)
    df[f'{column}_seasonal_shift_b_1m'] = df[f'{column}_seasonal'].shift(-1 * weeks_in_month)
    df[f'{column}_seasonal_shift_1m'] = df[f'{column}_seasonal'].shift(1 * weeks_in_month)
    df[f'{column}_seasonal_shift_2m'] = df[f'{column}_seasonal'].shift(2 * weeks_in_month)
    df[f'{column}_seasonal_shift_3m'] = df[f'{column}_seasonal'].shift(3 * weeks_in_month)

In [59]:
f, ax = plt.subplots(nrows=5, ncols=1, figsize=(15, 12))
f.suptitle('Seasonal Components of Features', fontsize=16)

for i, column in enumerate(core_columns):
    sns.lineplot(x=df['date'], y=df[column + '_seasonal'], ax=ax[i], color='dodgerblue', label='P25')
    ax[i].set_ylabel(ylabel=column, fontsize=14)
    ax[i].set_xlim([date(2017, 9, 30), date(2020, 6, 30)])
    
plt.tight_layout()
plt.show()

## Modelling

#### Split the data into Train and Test

In [60]:
# dividing the dataframe into train and valididation

train_size = int(0.85*len(df))
test_size = len(df)- train_size

univariate_df = df[['date', 'depth_to_groundwater']].copy()
univariate_df.columns = ['ds', 'y']

train = univariate_df.iloc[:train_size, :]

x_train,y_train = pd.DataFrame(univariate_df.iloc[:train_size,0]),pd.DataFrame(univariate_df.iloc[:train_size,1])
x_valid,y_valid = pd.DataFrame(univariate_df.iloc[train_size:,0]), pd.DataFrame(univariate_df.iloc[train_size:,1])

## FBProphet for Univariant timeseries

In [61]:
# Train the model
p_model = Prophet()
p_model.fit(train)

# Predict the y values
y_pred = p_model.predict(x_valid)

# calculate metrics

score_mae = mean_absolute_error(y_valid,y_pred['yhat'])
score_rmse = math.sqrt(mean_squared_error(y_valid,y_pred['yhat']))

print("\n")
print(f'RMSE: {score_rmse}')


In [62]:
f, ax = plt.subplots(1,figsize = (16,9))

p_model.plot(y_pred, ax = ax)
sns.lineplot(x=x_valid['ds'], y=y_valid['y'], ax = ax, color = 'orange',label = "True Y")

ax.set_title("Pred Y VS True Y")
ax.set_xlabel(xlabel='Date', fontsize=10)
ax.set_ylabel(ylabel='Depth to Groundwater', fontsize=10)

##### The task was to predict the depth of ground water for the future dates, In this above example we used phophet to forecast the future levels. But for this prediction we only used univariant to predict it.

##### that is, we used only depth of groundwater to forecast future levels.

## ARIMA Model 

In [63]:
df['depth_to_groundwater_first_diff'] = df['depth_to_groundwater'] - df['depth_to_groundwater'].shift(1)
df['depth_to_groundwater_first_diff'].iloc[0] = 0.0

In [64]:
autocorrelation_plot(df['depth_to_groundwater_first_diff'])
plt.show()

In [65]:
autocorrelation_plot(df['depth_to_groundwater_first_diff'])
plt.show()

In [66]:
A_model = ARIMA(y_train, order = (4,1,12))
model_fit = A_model.fit()
pred = model_fit.forecast(90)
df["pred Arima"] = pred

mse_arima = math.sqrt(mean_squared_error(y_valid,pred))
print(mse_arima)

In [67]:
# Predictions
pred

In [68]:
f, ax = plt.subplots(1,figsize = (16,10))

sns.lineplot(x = x_train['ds'], y = y_train['y'], ax= ax)
sns.lineplot(x = x_valid['ds'], y = y_valid['y'], ax= ax)
sns.lineplot(x = x_valid['ds'], y = pred, ax= ax)

## LSTM Model

In [69]:
dataset = univariate_df['y'].values.reshape(-1,1)

In [70]:
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(dataset)

In [71]:
train_size = int(len(dataset)*.67)
test_size = len(dataset)- train_size

train,test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(dataset),len(train),len(test))

In [72]:
def create_dataset(dataset, look_back):
    
    dataX,dataY = [],[]
    
    for i in range(len(dataset)-look_back-1):
        
        dataX.append(dataset[i:i+look_back,0])
        dataY.append(dataset[i+look_back,0])
    return np.array(dataX), np.array(dataY)    

In [73]:
X_train,Y_train = create_dataset(train,52)
X_test, Y_test = create_dataset(test,52)

In [74]:
X_train = np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))
X_test = np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))


In [75]:
model = Sequential()

model.add(LSTM(4,input_shape = (1,52)))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mse')
model.fit(X_train,Y_train,epochs = 60,verbose = 2, batch_size = 10, validation_data = (X_test,Y_test))

In [76]:

train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

train_predict = scaler.inverse_transform(train_predict)
Y_train = scaler.inverse_transform([Y_train])
test_predict = scaler.inverse_transform(test_predict)
Y_test = scaler.inverse_transform([Y_test])

print(math.sqrt(mean_squared_error(Y_train[0],train_predict[:,0])))
print(math.sqrt(mean_squared_error(Y_test[0],test_predict[:,0])))

In [77]:
test_predict.shape

In [78]:
univariate_df["ds"].tail(145).shape

In [79]:

f, ax = plt.subplots(figsize = (15,9))
sns.lineplot(x = univariate_df["ds"],y = univariate_df["y"], ax = ax)
sns.lineplot(x = univariate_df["ds"].tail(145) ,y = test_predict[:,0], ax = ax)

## Conclusion

##### LSTM gave the best result to forecast the future groundwater lavels compared to ARIMA model and FBprophet.